In [1]:
import tensorflow as tf
from gene_expression import *
from pathway_hierarchy import *
from utils import *
import torch

2024-08-03 12:28:29.671862: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-03 12:28:29.681246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 12:28:29.693278: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 12:28:29.693297: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-03 12:28:29.702288: I tensorflow/core/platform/cpu_feature_gua

In [44]:
import os
import torch
from tqdm import tqdm
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
import argparse
from utils import *
from gene_expression import *
from pathway_hierarchy import *
import pandas as pd
import yaml
from custom_neural_network import *
from custom_fc_network import *
from datetime import datetime
import csv
import copy
import pickle
import random
random.seed(0)
np.random.seed(0)


model_dct = dict()

# Hook function
def hook_fn(module, input, output, layer_name):
    global model_dct
    input_list = [i.detach().cpu().numpy().tolist() for i in input]
    output_list = output.detach().cpu().numpy().tolist()
    
    # If the layer name is not in the dictionary, create a new list for it
    if layer_name not in model_dct:
        model_dct[layer_name] = []

    # Append the activations to the corresponding layer list
    model_dct[layer_name].append({
        'input': input_list,
        'output': output_list
    })



# Define the file path for the CSV file
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

class TabularDataset(Dataset):
    def __init__(self, count_matrix, label):
        # Read the CSV file
        self.data = count_matrix
        # Separate features and target
        self.features = self.data.values
        self.target = label.values
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get features and target for a given index
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.target[idx], dtype=torch.float32)
        return features, target

def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    predicted_list = []
    probability_list = []
    labels_list = []
    criterion = nn.BCEWithLogitsLoss()
    loss = 0
    with torch.no_grad():  # No need to compute gradients during evaluation
        for features, labels in dataloader:
            outputs = model(features)
            #print(outputs)
            probability = torch.sigmoid(outputs.data)
            predicted = torch.round(torch.sigmoid(outputs.data))
            #print(outputs)
            #print(predicted)
            loss += criterion(outputs, labels)
            #_, predicted = torch.sigmoid(outputs.data)
            predicted_list.extend(predicted)
            labels_list.extend(labels)
            probability_list.extend(probability)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    #print(total)
    accuracy = 100 * correct / total
    return accuracy, loss, predicted_list, labels_list, probability_list

def save_model(model_nn,model_path, model_state_dict_path):
    
    model_nn.eval()
    torch.save(model_nn, model_path)
    torch.save(model_nn.state_dict(), model_state_dict_path)




def model_fc(train_dataloader , val_dataloader, test_dataloader, test_cell_id, layers_node, masking, output_layer,model_save_dir, date_string, learning_rate=0.001, num_epochs=50, weight_decay = 0):

    model_nn = CustomfcNetwork(layers_node, output_layer, masking)
    optimizer = optim.AdamW(model_nn.parameters(), lr=learning_rate,weight_decay = weight_decay )  # Using SGD with momentum
    criterion = nn.BCEWithLogitsLoss()
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)
    patience = 20
    best_val_accuracy = 0.0
    epochs_no_improve = 0
    early_stop = False
    csv_file_path = f'{model_save_dir}{date_string}/fc_training_log_{output_layer}.csv'

    try:
        os.makedirs(f'{model_save_dir}{date_string}')
    except:
        print(('...'))

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Epoch', 'Train_Loss', 'Train_accuracy','Validation_Loss','Val_accuracy'])

    for epoch in tqdm(range(num_epochs)):
        if early_stop:
            print("Early stopping")
            break
        epoch_cost = 0.
        
        total_loss = 0
        for batch_features,batch_targets in train_dataloader:
            outputs = model_nn(batch_features)
            #print(outputs)
            #print(batch_targets)
            #print(outputs)
            loss = criterion(outputs, batch_targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            
        
        train_accuracy, train_loss, predicted_list_train, labels_list_train, train_probability_list = evaluate(model_nn, train_dataloader)
        val_accuracy, val_loss, predicted_list_val, labels_list_val, val_probability_list = evaluate(model_nn, val_dataloader)
        #scheduler.step(val_accuracy)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item():.4f}, Train_accuracy: {train_accuracy}, Val Loss: {val_loss.item():.4f}, Val_accuracy: {val_accuracy}')
        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([epoch + 1, loss.item(), train_accuracy, val_loss.item(), val_accuracy])
        
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            epochs_no_improve = 0
        # Save the best model
            model_path = f'{model_save_dir}{date_string}/fc_best_model_{output_layer}.pth'
            model_state_dict_path = f'{model_save_dir}{date_string}/fc_best_model_{output_layer}_state_dict.pth'
            save_model(model_nn, model_path, model_state_dict_path)
            best_model_nn = copy.deepcopy(model_nn)
            #torch.save(model_nn, f'{model_save_dir}{date_string}/fc_best_model_{output_layer}.pth')
            #torch.save(model_nn.state_dict(), f'{model_save_dir}{date_string}/fc_best_model_{output_layer}_state_dict.pth')
            print('Model saved.')
        else:
            epochs_no_improve += 1
    
        # Early stopping
        '''if epochs_no_improve >= patience:
            early_stop = True
            print("Early stopping triggered")'''
        
    
    train_accuracy, train_loss, predicted_list_train, labels_list_train, train_probability_list = evaluate(best_model_nn, train_dataloader)
    val_accuracy, val_loss, predicted_list_val, labels_list_val, val_probability_list = evaluate(best_model_nn, val_dataloader)
    test_accuracy, test_loss, predicted_list_test, labels_list_test, test_probability_list = evaluate(best_model_nn, test_dataloader)
    print('Test Accucary', test_accuracy)
    output_train = (predicted_list_train, labels_list_train)
    output_val = (predicted_list_val, labels_list_val)

    labels_list_test = [m.item() for m in labels_list_test]
    predicted_list_test = [m.item() for m in predicted_list_test]
    test_probability_list = [m.item() for m in test_probability_list]


    test_df = pd.DataFrame({'cell_id': test_cell_id, 'true_y': labels_list_test, 'pred_y': predicted_list_test, 'probabilty': test_probability_list})
    csv_file_path = f'{model_save_dir}{date_string}/fc_test_log_{output_layer}.csv'
    test_df.to_csv(csv_file_path)
    #torch.save(model_nn, f'{model_save_dir}{date_string}/fc_last_epoch_model_{output_layer}.pth')
    return output_train, output_val,best_model_nn



def model(train_dataloader , val_dataloader, test_dataloader, test_cell_id, layers_node, masking, output_layer,model_save_dir, date_string, learning_rate=0.001, num_epochs=50, weight_decay = 0):

    model_nn = CustomNetwork(layers_node, output_layer, masking)
    optimizer = optim.AdamW(model_nn.parameters(), lr=learning_rate,weight_decay = weight_decay )  # Using SGD with momentum
    criterion = nn.BCEWithLogitsLoss()
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)
    patience = 20
    best_val_accuracy = 0.0
    epochs_no_improve = 0
    early_stop = False
    csv_file_path = f'{model_save_dir}{date_string}/training_log_{output_layer}.csv'

    try:
        os.makedirs(f'{model_save_dir}{date_string}')
    except:
        print(('...'))

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Epoch', 'Train_Loss', 'Train_accuracy','Validation_Loss','Val_accuracy'])

    for epoch in tqdm(range(num_epochs)):
        if early_stop:
            print("Early stopping")
            break
        epoch_cost = 0.
        
        total_loss = 0
        for batch_features,batch_targets in train_dataloader:
            
            #print(outputs)
            #print(batch_targets)
            #print(outputs)
            
            
            optimizer.zero_grad()
            outputs = model_nn(batch_features)
            loss = criterion(outputs, batch_targets)
            loss.backward()
            optimizer.step()
            
            
        
        train_accuracy, train_loss, predicted_list_train, labels_list_train, train_probability_list = evaluate(model_nn, train_dataloader)
        val_accuracy, val_loss, predicted_list_val, labels_list_val, val_probability_list = evaluate(model_nn, val_dataloader)
        #scheduler.step(val_accuracy)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item():.4f}, Train_accuracy: {train_accuracy}, Val Loss: {val_loss.item():.4f}, Val_accuracy: {val_accuracy}')
        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([epoch + 1, loss.item(), train_accuracy, val_loss.item(), val_accuracy])
        
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            epochs_no_improve = 0
        # Save the best model
            model_path = f'{model_save_dir}{date_string}/best_model_{output_layer}.pth'
            model_state_dict_path = f'{model_save_dir}{date_string}/best_model_{output_layer}_state_dict.pth'
            save_model(model_nn, model_path, model_state_dict_path)
            best_model_nn = copy.deepcopy(model_nn)
            #torch.save(model_nn, f'{model_save_dir}{date_string}/best_model_{output_layer}.pth')
            #torch.save(model_nn.state_dict(), f'{model_save_dir}{date_string}/best_model_{output_layer}_state_dict.pth')
            print('Model saved.')
        else:
            epochs_no_improve += 1
    
        # Early stopping
        '''if epochs_no_improve >= patience:
            early_stop = True
            print("Early stopping triggered")'''
        
    
    train_accuracy, train_loss, predicted_list_train, labels_list_train, train_probability_list = evaluate(best_model_nn, train_dataloader)
    val_accuracy, val_loss, predicted_list_val, labels_list_val, val_probability_list = evaluate(best_model_nn, val_dataloader)
    test_accuracy, test_loss, predicted_list_test, labels_list_test, test_probability_list = evaluate(best_model_nn, test_dataloader)
    print('Test Accucary', test_accuracy)
    output_train = (predicted_list_train, labels_list_train)
    output_val = (predicted_list_val, labels_list_val)

    labels_list_test = [m.item() for m in labels_list_test]
    predicted_list_test = [m.item() for m in predicted_list_test]
    test_probability_list = [m.item() for m in test_probability_list]


    test_df = pd.DataFrame({'cell_id': test_cell_id, 'true_y': labels_list_test, 'pred_y': predicted_list_test, 'probabilty': test_probability_list})
    csv_file_path = f'{model_save_dir}{date_string}/test_log_{output_layer}.csv'
    test_df.to_csv(csv_file_path)
    #torch.save(model_nn, f'{model_save_dir}{date_string}/last_epoch_model_{output_layer}.pth')
    return output_train, output_val,best_model_nn


def load_config(config_file):
    with open(config_file, 'r') as file:
        return yaml.safe_load(file)


def main_file():

    '''parser = argparse.ArgumentParser(description='Sample application with config and argparse')
    parser.add_argument('--config', type=str, default='config.yml', help='Path to the configuration file')
    args = parser.parse_args()'''

    config = load_config('config.yml')
    train = pd.read_csv(config['dataset']['train'],index_col=0)
    test = pd.read_csv(config['dataset']['test'],index_col=0)
    val = pd.read_csv(config['dataset']['val'],index_col=0)

    y_train = pd.read_csv(config['dataset']['y_train'])
    y_test = pd.read_csv(config['dataset']['y_test'])
    y_val = pd.read_csv(config['dataset']['y_val'])
  


    r_data_tmp = train.T
    q_data_tmp = test.T
    v_data_tmp = val.T
    r_label_tmp = y_train

    print('Getting Marker Genes.......')
    train_x, test_x, val_x, train_y = get_expression(r_data_tmp,
                                                q_data_tmp,
                                                v_data_tmp,
                                                r_label_tmp,
                                                thrh=config['gene_expression']['highly_expressed_threshold'],
                                                thrl=config['gene_expression']['lowly_expressed_threshold'],
                                                normalization=config['gene_expression']['normalization'],
                                                marker=config['gene_expression']['marker'])
    
    print('Getting Pathway Genes.........')
    pathway_genes = get_gene_pathways(config['pathways_network']['ensemble_pathway_relation'], species=config['pathways_network']['species'])


    print('Getting Masking.........')
    masking, masking_df, layers_node, train_x, test_x,val_x = get_masking(config['pathways_network']['pathway_names'],
                                                        pathway_genes,
                                                        config['pathways_network']['pathway_relation'],
                                                        train_x,
                                                        test_x,
                                                        val_x,
                                                        train_y,
                                                        config['pathways_network']['datatype'],
                                                        config['pathways_network']['species'],
                                                        config['pathways_network']['n_hidden_layer'])

    test_cell_id = list(test_x.T.index) 
    try:
        masking = list(masking.values())
        layers_node = list(layers_node.values())
    except:
        print('already_done')


    train_dataset = TabularDataset(train_x.T,train_y)
    val_dataset = TabularDataset(val_x.T,y_val)
    test_dataset = TabularDataset(test_x.T,y_test)  
    
    

    dataloader_params = {
    'batch_size': config['train']['batch_size'],
    'shuffle': False
    }

    train_dataloader = DataLoader(train_dataset,**dataloader_params)
    test_dataloader = DataLoader(test_dataset, **dataloader_params)
    val_dataloader = DataLoader(val_dataset,**dataloader_params)
    # Example of iterating through the DataLoader


    pred_y_df = pd.DataFrame(data=0, index=test_x.columns, columns=list(range(2, len(masking) + 2)))
    train_y_df = pd.DataFrame(data=0, index=train_x.columns, columns=list(range(2, len(masking) + 2)))
    model_dict_sparse = dict()
    model_dict_fc = dict()
    activation_output = {}
    now = datetime.now()

# Format the date as a string
    date_string = datetime_string = now.strftime("%Y_%m_%d_%H_%M_%S")

    try:
        os.makedirs(f'{config['model_output']['model_save_dir']}{date_string}')
    except:
        print(('...'))

   

    print('Training.........')
    for output_layer in range(2, len(masking) + 2):
        if config['gene_expression']['print_information']:
            print("Current sub-neural network has " + str(output_layer - 1) + " hidden layers.")
        output_train, output_val,model_dict_sparse[output_layer] = model(train_dataloader,
                                            val_dataloader,test_dataloader, test_cell_id,
                                            layers_node,
                                            masking,
                                            output_layer,
                                            model_save_dir = config['model_output']['model_save_dir'],date_string = date_string,
                                            learning_rate=config['train']['learning_rate'],num_epochs=config['train']['epochs'],weight_decay = config['train']['weight_decay']
                                        )  

    print('tranining_fully_connected_layers:')
    for output_layer in range(2, len(masking) + 2):
        if config['gene_expression']['print_information']:
            print("Current sub-neural network has " + str(output_layer - 1) + " hidden layers.")
        output_train, output_val,model_dict_fc[output_layer] = model_fc(train_dataloader,
                                            val_dataloader,test_dataloader, test_cell_id,
                                            layers_node,
                                            masking,
                                            output_layer,
                                            model_save_dir = config['model_output']['model_save_dir'],date_string = date_string,
                                            learning_rate=config['train']['learning_rate'],num_epochs=config['train']['epochs'],weight_decay = config['train']['weight_decay']
                                        )  
        
    new_parameter = {'date_string': date_string}
    config.update(new_parameter)
    save_path =   str(config['model_output']['model_save_dir'])+ date_string + '/config.yml'
    with open(save_path, 'w') as file:
        yaml.dump(config, file)

        
    for i in range(len(masking_df)):
        masking_df[i].to_csv(str(config['model_output']['model_save_dir'])+ date_string+ '/' +f'masking_df_{i}.csv')
    
    dataloader_params = {
    'batch_size': 1,
    'shuffle': False
    }

    train_dataloader = DataLoader(train_dataset,**dataloader_params)
    test_dataloader = DataLoader(test_dataset, **dataloader_params)
    val_dataloader = DataLoader(val_dataset,**dataloader_params)
    
    for j,i in model_dict_sparse.items():
        
        for name, layer in enumerate(i.children()):
            layer_name = 'fc'+str(name+1)
            layer.register_forward_hook(lambda module, input, output, name=layer_name: hook_fn(module, input, output, name))



        accuracy, loss, predicted_list, labels_list, probability_list = evaluate(i, test_dataloader)
        print(j)
        print(f'Test Accuracy: {accuracy}')   
        accuracy, loss, predicted_list, labels_list, probability_list = evaluate(i, train_dataloader)
        print(f'Train Accuracy: {accuracy}')   
        accuracy, loss, predicted_list, labels_list, probability_list = evaluate(i, val_dataloader)
        print(f'Validation Accuracy: {accuracy}') 

        '''with open('model_activations_train_test.csv', 'w', newline='') as csvfile:
            fieldnames = ['layer', 'input', 'output']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            for layer_name, activations in model_dct.items():
                for activation in activations:
                    writer.writerow({
                        'layer': layer_name,
                        'input': activation['input'],
                        'output': activation['output']
                    })'''
        
        break
        
    return model_dict_sparse, test_dataloader, train_dataloader, config

   


                                            


                            

In [45]:
model_dict_sparse, test_dataloader, train_dataloader, config = main_file()

Getting Marker Genes.......
1125
1125
2250
2250
                    0                1
0     ENSG00000101210  ENSG00000172270
1     ENSG00000099622  ENSG00000141905
2     ENSG00000105278  ENSG00000167658
3     ENSG00000178951  ENSG00000089847
4     ENSG00000141985  ENSG00000127663
...               ...              ...
1120  ENSG00000285395  ENSG00000103316
1121  ENSG00000140740  ENSG00000284218
1122  ENSG00000122254  ENSG00000103365
1123  ENSG00000006116  ENSG00000182601
1124  ENSG00000077235  ENSG00000171208

[1125 rows x 2 columns]
                    0                1
0     ENSG00000101210  ENSG00000172270
1     ENSG00000099622  ENSG00000141905
2     ENSG00000105278  ENSG00000167658
3     ENSG00000178951  ENSG00000089847
4     ENSG00000141985  ENSG00000127663
...               ...              ...
1120  ENSG00000285395  ENSG00000103316
1121  ENSG00000140740  ENSG00000284218
1122  ENSG00000122254  ENSG00000103365
1123  ENSG00000006116  ENSG00000182601
1124  ENSG00000077235  ENSG000

/home/anwer/.local/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Training.........
Current sub-neural network has 1 hidden layers.
...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.72it/s]

Epoch [1/1], Train Loss: 4.8215, Train_accuracy: 55.59815406460774, Val Loss: 2.0674, Val_accuracy: 55.484558040468585
Model saved.


...

Epoch [1/1], Train Loss: 4.5107, Train_accuracy: 61.49094781682641, Val Loss: 1.9456, Val_accuracy: 60.298189563365284
Model saved.


Test Accucary 60.83901192504259
Current sub-neural network has 3 hidden layers.
...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.69it/s]

Epoch [1/1], Train Loss: 4.2285, Train_accuracy: 69.20127795527156, Val Loss: 1.8249, Val_accuracy: 69.05218317358893
Model saved.


Test Accucary 68.99488926746167
2
Test Accuracy: 55.51533219761499
Train Accuracy: 55.59815406460774
Validation Accuracy: 55.484558040468585


In [46]:
for j,i in model_dict_sparse.items():
        
        '''for name, layer in enumerate(i.children()):
            layer_name = 'fc'+str(name+1)
            layer.register_forward_hook(lambda module, input, output, name=layer_name: hook_fn(module, input, output, name))'''



        accuracy, loss, predicted_list, labels_list, probability_list = evaluate(i, test_dataloader)
        print(j)
        print(f'Test Accuracy: {accuracy}')   
        accuracy, loss, predicted_list, labels_list, probability_list = evaluate(i, train_dataloader)
        print(f'Train Accuracy: {accuracy}')   
        '''accuracy, loss, predicted_list, labels_list, probability_list = evaluate(i, val_dataloader)
        print(f'Validation Accuracy: {accuracy}') '''

2
Test Accuracy: 55.51533219761499
Train Accuracy: 55.59815406460774
3
Test Accuracy: 50.27683134582624
Train Accuracy: 49.82605608803692
4
Test Accuracy: 50.27683134582624
Train Accuracy: 49.82605608803692


In [12]:
model_dict_sparse

{2: CustomNetwork(
   (layers): ModuleList(
     (0): Linear(in_features=582, out_features=356, bias=False)
     (1): Linear(in_features=356, out_features=1, bias=True)
   )
 ),
 3: CustomNetwork(
   (layers): ModuleList(
     (0): Linear(in_features=582, out_features=356, bias=False)
     (1): Linear(in_features=356, out_features=134, bias=False)
     (2): Linear(in_features=134, out_features=1, bias=True)
   )
 ),
 4: CustomNetwork(
   (layers): ModuleList(
     (0): Linear(in_features=582, out_features=356, bias=False)
     (1): Linear(in_features=356, out_features=134, bias=False)
     (2): Linear(in_features=134, out_features=29, bias=False)
     (3): Linear(in_features=29, out_features=1, bias=True)
   )
 )}

In [13]:
config

{'dataset': {'train': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/Preprocessed_data/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/train.csv',
  'test': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/Preprocessed_data/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/test.csv',
  'val': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/Preprocessed_data/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/val.csv',
  'y_train': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/Preprocessed_data/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/y_train.csv',
  'y_test': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/Preprocessed_data/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/y_test.csv',
  'y_val': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/Preprocessed_data/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/y_val.csv'},
 'model_output': {'model_save_dir': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/model_

In [15]:
config['model_output']['model_save_dir']+ config['date_string']+'/'

'/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/model_save/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/2024_08_04_09_14_03/'

In [31]:
for j,i in model_dict_sparse.items():

# Assuming 'model' is your neural network
    torch.save(i.state_dict(), f'{config['model_output']['model_save_dir']}{config['date_string']}/model_{j}_state_dict_jupyter_notebook.pth')

In [47]:
import io
#buffer = io.BytesIO(model_data)

model_2= torch.load('/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/model_save/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/2024_08_04_09_14_03//model_2_state_dict_jupyter_notebook.pth')

In [48]:
model_2

OrderedDict([('layers.0.weight',
              tensor([[-0., 0., 0.,  ..., -0., -0., 0.],
                      [-0., -0., 0.,  ..., -0., -0., 0.],
                      [-0., 0., 0.,  ..., 0., -0., 0.],
                      ...,
                      [-0., 0., 0.,  ..., 0., -0., 0.],
                      [0., -0., -0.,  ..., -0., 0., -0.],
                      [-0., -0., 0.,  ..., -0., 0., -0.]])),
             ('layers.1.weight',
              tensor([[-1.3116e-01, -1.0516e-01, -6.4996e-02, -2.5069e-01,  1.8438e-01,
                        3.8274e-02, -1.7296e-01,  3.2673e-03,  1.6031e-01, -2.3190e-02,
                        3.5430e-02, -1.7787e-01,  1.8825e-01, -6.6232e-03,  7.4582e-02,
                       -2.9358e-01, -4.3745e-03,  3.6318e-02, -2.4986e-01, -3.0154e-03,
                       -5.2295e-02, -9.7599e-02, -3.4880e-01,  8.7646e-03, -1.6985e-01,
                        3.1090e-02,  4.1819e-02, -2.0573e-01, -2.7015e-02,  1.3528e-02,
                        1.0335e-0

In [49]:
m = model_dict_sparse[2]
m.load_state_dict(model_2)

<All keys matched successfully>

In [50]:
accuracy, loss, predicted_list, labels_list, probability_list = evaluate(m , test_dataloader)
accuracy

75.74531516183987

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-07-31 14:11:36.224737: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-31 14:11:36.227061: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-31 14:11:36.227177: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
import pandas as pd
import numpy as np

In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2024-07-31 14:11:36.242744: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


True

2024-07-31 14:11:36.242868: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-31 14:11:36.242962: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-31 14:11:36.410684: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
torch.cuda.is_available()


True

In [6]:
rdata = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/train_test_set/train_count_matrix.csv', index_col=0)
qdata = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/train_test_set/test_count_matrix.csv', index_col=0)
rlabel = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/train_test_set/train_label.csv')
qlabel = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/train_test_set/test_label.csv')

In [ ]:
single_cell_metadata = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/metadata_inhibitory_neurons.csv')

In [ ]:
single_cell_metadata

In [ ]:
# Verifying Train Label is correct

list(single_cell_metadata.set_index('cell_id').loc[rdata.columns,].clinical_pathological_AD) == list(rlabel.diagnosis)

In [ ]:
# Verifying Test Label is correct

list(single_cell_metadata.set_index('cell_id').loc[qdata.columns,].clinical_pathological_AD) == list(qlabel.diagnosis)

In [ ]:
rdata = rdata.T
qdata = qdata.T
rdata

In [ ]:
rdata['diagnosis'] =  list(rlabel.diagnosis)
qdata['diagnosis'] =  list(qlabel.diagnosis)

In [ ]:
rdata.diagnosis.value_counts()

In [ ]:
rdata_neg = rdata[rdata.diagnosis == 'NCI_with_No_Plaques'].iloc[0:24372,]
rdata_pos = rdata[rdata.diagnosis == 'AD_with_Plaques'].iloc[0:24372,]
rdata = pd.concat([rdata_neg,rdata_pos])


In [ ]:
rdata

In [ ]:
rdata = rdata.sample(frac = 1)

In [ ]:
rdata

In [ ]:
rlabel = pd.DataFrame(rdata.diagnosis)

In [ ]:
rlabel

In [ ]:
rdata = rdata.drop(columns=['diagnosis'])

In [ ]:
qdata.diagnosis.value_counts()

In [ ]:
qdata_neg = qdata[qdata.diagnosis == 'NCI_with_No_Plaques'].iloc[0:3442,]
qdata_pos =qdata[qdata.diagnosis == 'AD_with_Plaques'].iloc[0:3442,]
qdata = pd.concat([qdata_neg,qdata_pos])

In [ ]:
qdata

In [ ]:
qdata = qdata.sample(frac = 1)

In [ ]:
qdata

In [ ]:
qlabel = pd.DataFrame(qdata.diagnosis)

In [ ]:
qlabel.reset_index(drop = True, inplace = True)
rlabel.reset_index(drop = True, inplace = True)
rlabel

In [ ]:
qdata = qdata.drop(columns=['diagnosis'])

In [ ]:
qdata

In [ ]:
rdata = rdata.T
qdata = qdata.T

In [ ]:
r_data_tmp = rdata.copy()
q_data_tmp = qdata.copy()
r_label_tmp = rlabel.copy()

In [ ]:
import numpy as np
import pandas as pd


def fphi(x):
  zt = (1 - (x + 1).div(x.max(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)
  zp = (x + 1).div(x.max(axis=1) + 1, axis=0).mul(zt, axis=0)
  return zp


def frho(x):
  zt = (1 - (1 / (x + 1)).mul(x.min(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)
  zp = (1 / (x + 1)).mul(x.min(axis=1) + 1, axis=0).mul(zt, axis=0)
  return zp


def fmkg(phi, rho, thr1=0.95, thr2=0.9):
  #print(phi)
  #print(rho)
  gnm = phi.index.values
  ctpnm = phi.columns.values
  phi = np.array(phi)
  rho = np.array(rho)
  nummkg1 = round((1 - thr1) * phi.shape[0])
  nummkg2 = round((1 - thr2) * phi.shape[0])
  print(nummkg1)
  print(nummkg2)
  alpha = []
  beta = []
  for i in range(0, phi.shape[1]):
    alpha = np.append(alpha, np.quantile(phi[:, i], thr1))
    beta = np.append(beta, np.quantile(rho[:, i], thr2))
  mkh = []
  mkl = []
  for i in range(0, phi.shape[1]):
    mkh = np.concatenate([mkh, gnm[phi[:, i] >= alpha[i]][0:nummkg1]], axis=0)
    mkl = np.concatenate([mkl, gnm[rho[:, i] >= beta[i]][0:nummkg2]], axis=0)
  
  print(len(mkh))
  print(len(mkl))
  mkh = mkh.reshape(nummkg1, phi.shape[1])
  mkl = mkl.reshape(nummkg2, rho.shape[1])
  mkh = pd.DataFrame(mkh, columns=ctpnm)
  mkl = pd.DataFrame(mkl, columns=ctpnm)
  print(mkh)
  print(mkl)
  return mkh, mkl


def get_expression(rdata, qdata, rlabel, thrh=0.95, thrl=0.9, normalization=True, marker=True):
    # calculate sum of cell type
    rulabel = rlabel.iloc[:, 0].unique()
    rdt = pd.DataFrame(data=None, columns=None)
    for l in rulabel:
      rdata_l = rdata.iloc[:, rlabel[(rlabel["diagnosis"] == l)].index.tolist()] #row value in rlabel, is column in rdata
      zs = rdata_l.apply(lambda x: x.sum(), axis=1)
      rdt = pd.concat([rdt, pd.DataFrame(data=zs, columns=[l])], axis=1)
      

    # normalization
    if normalization:
        rdt_df = rdt
        rdata_df = rdata
        qdata_df = qdata
        rdt = np.array(rdt_df, dtype=np.float32)
        rdata = np.array(rdata_df, dtype=np.float32)
        qdata = np.array(qdata_df, dtype=np.float32)
        rdt = np.divide(rdt, np.sum(rdt, axis=0, keepdims=True)) * 10000
        rdata = np.log2(np.divide(rdata, np.sum(rdata, axis=0, keepdims=True)) * 10000 + 1)
        qdata = np.log2(np.divide(qdata, np.sum(qdata, axis=0, keepdims=True)) * 10000 + 1)

        rdt = pd.DataFrame(data=rdt, columns=rdt_df.columns, index=rdt_df.index)
        rdata = pd.DataFrame(data=rdata, columns=rdata_df.columns, index=rdata_df.index)
        qdata = pd.DataFrame(data=qdata, columns=qdata_df.columns, index=qdata_df.index)
    #print(rdt.head())
    #print(rdata.head())
    # match gene ensembl ids between the query data and the reference data
    nm = qdata.index.tolist()
    zg = pd.DataFrame(data=nm, columns=['gene'])
    gid = []
    for i in range(0, len(nm)):
      if nm[i] not in rdata.index.tolist():
        zg.iloc[i] = ''
      else:
        gid = gid + [nm[i]]
    qdata = qdata.loc[gid, :]
    rdata = rdata.loc[gid, :]
    rdt = rdt.loc[gid, :]

    rlabel.index = rdata.columns

    if not marker:
        train_x = rdata
        train_y = rlabel
        test_x = qdata
    else:
        # calculate important genes
        phi = fphi(rdt)
        rho = frho(rdt)
        mkh, mkl = fmkg(phi, rho, thrh, thrl)
        mkg = np.unique(np.append(np.array(mkh).reshape(1, -1), np.array(mkl).reshape(1, -1)))

        # return training set and testing set
        train_x = rdata.loc[mkg, :]
        train_y = rlabel
        test_x = qdata.loc[mkg, :]
    return train_x, test_x, train_y


In [ ]:

r_data_tmp = rdata.copy()
q_data_tmp = qdata.copy()
r_label_tmp = rlabel.copy()
train_x, test_x, train_y = get_expression(r_data_tmp,
                                              q_data_tmp,
                                              r_label_tmp,
                                              thrh=0.90,
                                              thrl=0.90,
                                              normalization=True,
                                              marker=True)

In [ ]:
train_x

In [ ]:
train_y

In [ ]:
train_y.diagnosis.value_counts()

In [ ]:
test_x

In [ ]:
ensembl_pathway_relation = '../../usman/CellTICS/reactome/Ensembl2Reactome_All_Levels.txt'
species = 'human'
pathway_names = '../../usman/CellTICS/reactome/ReactomePathways.txt'
pathway_relation = '../../usman/CellTICS/reactome/ReactomePathwaysRelation.txt'
datatype = 'diagnosis'
n_hidden = 5

In [ ]:
pathway_genes = get_gene_pathways(ensembl_pathway_relation, species=species)

In [ ]:
#set(pathway_genes.gene)-set(test_x.index)

In [ ]:
masking, layers_node, train_x, test_x = get_masking(pathway_names,
                                                        pathway_genes,
                                                        pathway_relation,
                                                        train_x,
                                                        test_x,
                                                        train_y,
                                                        datatype,
                                                        species,
                                                        n_hidden)

In [ ]:
masking[2].shape

In [ ]:
masking[2]

In [ ]:
train_x

In [ ]:
test_x

In [ ]:
#pd.DataFrame(test_x.index).to_csv('../../preprocessed_data/inhibitory_neuron/marker_genes.csv', index=False)

In [ ]:
train_x.T

In [ ]:
train_y

In [ ]:
diagnosis_mapping = {'AD_with_Plaques': 1, 'NCI_with_No_Plaques': 0}
train_y['diagnosis_binary'] = train_y['diagnosis'].map(diagnosis_mapping)

# Display the first few rows to verify the mapping
print(train_y.head())
train_y.drop(columns=['diagnosis'], inplace=True)
train_y

In [ ]:
qlabel

In [ ]:
try:
    diagnosis_mapping = {'AD_with_Plaques': 1, 'NCI_with_No_Plaques': 0}
    qlabel['diagnosis_binary'] = qlabel['diagnosis'].map(diagnosis_mapping)
    
    # Display the first few rows to verify the mapping
    print(qlabel.head())
    qlabel.drop(columns=['diagnosis'], inplace=True)
    qlabel

except:
    print('Previously Run Before')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class TabularDataset(Dataset):
    def __init__(self, count_matrix, label):
        # Read the CSV file
        self.data = count_matrix
        # Separate features and target
        self.features = self.data.values
        self.target = label.values
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get features and target for a given index
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.target[idx], dtype=torch.float32)
        return features, target

# Example usage



In [ ]:
qlabel

In [ ]:
train_y

In [ ]:
val_x = train_x.T.iloc[40000:,]
val_y = train_y.iloc[40000:,]
train_x = train_x.T.iloc[0:40000,]
train_y = train_y.iloc[0:40000,]


In [ ]:
train_dataset = TabularDataset(train_x,train_y)
val_dataset = TabularDataset(val_x,val_y)
test_dataset = TabularDataset(test_x.T,qlabel)

In [ ]:
val_x

In [ ]:
qlabel

In [ ]:
masking[0].shape, len(layers_node[1])

In [ ]:
masking[1].shape, len(layers_node[2])

In [ ]:
masking[2].shape, len(layers_node[3])

In [ ]:
masking = list(masking.values())
layers_node = list(layers_node.values())

In [ ]:
 import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


#for output_layer in range(2, len(masking) + 2):       -->n
class CustomNetwork(nn.Module):
    def __init__(self, layers_node, n, masking):
        super(CustomNetwork, self).__init__()
        self.layers = nn.ModuleList()
        self.masks = []
        self.gamma = 0.0001

        for i in range(n-1):
            #print(-1-i)
            #print(layers_node[-1-i])
            in_features = len(layers_node[-1-i])
            out_features = len(layers_node[-1-i-1])
            self.layers.append(nn.Linear(in_features, out_features, bias=False))
            self.masks.append(masking[-1-i])

        self.layers.append(nn.Linear(len(layers_node[-n]), len(layers_node[0])-1))
        self.masks = [torch.tensor(mask, dtype=torch.float32) for mask in self.masks]


    def forward(self, x):
        for i, layer in enumerate(self.layers[:-1]):
            mask = self.masks[i]
            #print(mask[0].shape)
            layer.weight.data *= mask
            #print(layer(x).shape)
            x = layer(x)
            
            '''
            x = x.unsqueeze(2)
            print('x shape before multi: ',x.shape)
            print('shape of mask: ', mask.shape )
            x =  x*mask
            x = x.squeeze(2)
            x = torch.tanh(x)
            '''
            x = torch.relu(x)
            #print('x shape: ',x.shape)
        #print(x.shape)
        
        x = self.layers[-1](x)
        return x


In [ ]:
def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    predicted_list = []
    labels_list = []
    
    with torch.no_grad():  # No need to compute gradients during evaluation
        for features, labels in dataloader:
            outputs = model(features)
            #print(outputs)
            predicted = torch.round(torch.sigmoid(outputs.data))
            #print(outputs)
            #print(predicted)
            #_, predicted = torch.sigmoid(outputs.data)
            predicted_list.append(predicted)
            labels_list.append(labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    #print(total)
    accuracy = 100 * correct / total
    return accuracy, predicted_list, labels_list

In [ ]:
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

def model(train_dataloader , val_dataloader, layers_node, masking, output_layer, learning_rate=0.001, num_epochs=50):

    model = CustomNetwork(layers_node, output_layer, masking)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Using SGD with momentum
    criterion = nn.BCEWithLogitsLoss()
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)
    patience = 20
    best_val_accuracy = 0.0
    epochs_no_improve = 0
    early_stop = False
    
    for epoch in tqdm(range(num_epochs)):
        if early_stop:
            print("Early stopping")
            break
        epoch_cost = 0.
        
        for batch_features,batch_targets in train_dataloader:
            outputs = model(batch_features)
            #print(outputs)
            #print(batch_targets)
            #print(outputs)
            loss = criterion(outputs, batch_targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        
        train_accuracy, predicted_list_train, labels_list_train = evaluate(model, train_dataloader)
        val_accuracy, predicted_list_val, labels_list_val = evaluate(model, val_dataloader)
        scheduler.step(val_accuracy)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Train_accuracy: {train_accuracy}, Val_accuracy: {val_accuracy}')

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            epochs_no_improve = 0
        # Save the best model
            torch.save(model.state_dict(), f'best_model_{output_layer}.pth')
            print('Model saved.')
        else:
            epochs_no_improve += 1
    
        # Early stopping
        if epochs_no_improve >= patience:
            early_stop = True
            print("Early stopping triggered")
        
    
    train_accuracy, predicted_list_train, labels_list_train = evaluate(model, train_dataloader)
    val_accuracy, predicted_list_val, labels_list_val = evaluate(model, val_dataloader)
    
    output_train = (predicted_list_train, labels_list_train)
    output_val = (predicted_list_val, labels_list_val)
    
    return output_train, output_val,model

In [ ]:
torch.round(torch.sigmoid(torch.tensor([0.4,0.8])))

In [ ]:

#from neural_network import *
print_information = True
num_epochs = 500
learning_rate = 0.01
batch_size = 2048
l2_regularization = 0.00001
print_cost = True


# Create the DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle= True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle= True)
# Example of iterating through the DataLoader


pred_y_df = pd.DataFrame(data=0, index=test_x.columns, columns=list(range(2, len(masking) + 2)))
train_y_df = pd.DataFrame(data=0, index=train_x.columns, columns=list(range(2, len(masking) + 2)))
#masking = list(masking.values())
model_dict = dict()
#layers_node = list(layers_node.values())
activation_output = {}
for output_layer in range(2, len(masking) + 2):
    if print_information:
        print("Current sub-neural network has " + str(output_layer - 1) + " hidden layers.")
    output_train, output_test,model_dict[output_layer] = model(train_dataloader,
                                          val_dataloader,
                                          layers_node,
                                          masking,
                                          output_layer,
                                          learning_rate=learning_rate,num_epochs=num_epochs
                                          )

In [ ]:
model_dict

In [ ]:
model_dict[2]

In [ ]:
model_dict[2].layers[0].weight.shape

In [124]:
masking[0].shape

(28, 135)

In [ ]:
print(model_dict)

In [ ]:
!pip install wandb -Uq

In [ ]:
pred_y_df

In [ ]:
model_dict[2].layers[0].weight

In [84]:
masking[2].shape

(390, 1242)

In [91]:
model_dict[2].layers[0].weight

Parameter containing:
tensor([[0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, 0.0000],
        [-0.0000, 0.0000, -0.0000,  ..., -0.0000, 0.0000, 0.0000],
        [-0.0000, -0.0000, 0.0000,  ..., -0.0000, -0.0000, -0.0000],
        ...,
        [-0.0000, 0.0674, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, -0.0000, -0.0000]],
       requires_grad=True)

In [106]:
torch.nonzero(torch.tensor(model_dict[4].layers[2].weight)).squeeze()

/tmp/ipykernel_888033/1061644972.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.nonzero(torch.tensor(model_dict[4].layers[2].weight)).squeeze()


tensor([[  0,  27],
        [  0,  33],
        [  0,  37],
        [  0,  42],
        [  0,  98],
        [  0, 100],
        [  0, 112],
        [  0, 127],
        [  1,   8],
        [  1,  23],
        [  1,  25],
        [  1,  32],
        [  1,  44],
        [  1,  53],
        [  1,  56],
        [  1,  73],
        [  1,  83],
        [  1,  92],
        [  1,  93],
        [  1,  97],
        [  1, 105],
        [  1, 107],
        [  1, 129],
        [  1, 141],
        [  2, 106],
        [  3,   3],
        [  3,  13],
        [  3,  18],
        [  3,  28],
        [  3,  41],
        [  3,  59],
        [  3,  71],
        [  3,  79],
        [  3,  95],
        [  3, 104],
        [  3, 110],
        [  3, 111],
        [  3, 126],
        [  3, 137],
        [  3, 138],
        [  4, 108],
        [  5,  31],
        [  5,  47],
        [  5,  54],
        [  5,  55],
        [  5,  63],
        [  5,  68],
        [  5,  96],
        [  6,   7],
        [  6,  20],


In [107]:
non_zero_indices_flat = torch.nonzero(torch.tensor(masking[0])).squeeze()
non_zero_indices_flat 

tensor([[  0,  27],
        [  0,  33],
        [  0,  37],
        [  0,  42],
        [  0,  98],
        [  0, 100],
        [  0, 112],
        [  0, 127],
        [  1,   8],
        [  1,  23],
        [  1,  25],
        [  1,  32],
        [  1,  44],
        [  1,  53],
        [  1,  56],
        [  1,  73],
        [  1,  83],
        [  1,  92],
        [  1,  93],
        [  1,  97],
        [  1, 105],
        [  1, 107],
        [  1, 129],
        [  1, 141],
        [  2, 106],
        [  3,   3],
        [  3,  13],
        [  3,  18],
        [  3,  28],
        [  3,  41],
        [  3,  59],
        [  3,  71],
        [  3,  79],
        [  3,  95],
        [  3, 104],
        [  3, 110],
        [  3, 111],
        [  3, 126],
        [  3, 137],
        [  3, 138],
        [  4, 108],
        [  5,  31],
        [  5,  47],
        [  5,  54],
        [  5,  55],
        [  5,  63],
        [  5,  68],
        [  5,  96],
        [  6,   7],
        [  6,  20],


In [ ]:
'''
    train_y_df[output_layer] = get_prediction(output_train[output_layer],
                                                 pd.get_dummies(train_y['diagnosis']),
                                                 train_x,
                                                 datatype='diagnosis')

    
    for j in range(len(output_train)):
        ctp_sort = layers_node[0]
        ctp_sort.sort()
        if j != output_layer - 1:
            output_train[j + 1] = pd.DataFrame(data=output_train[j + 1],
                                                   index=layers_node[len(layers_node) - 2 - j],
                                                   columns=train_x.columns)
        else:
            output_train[j + 1] = pd.DataFrame(data=output_train[j + 1],
                                                   index=ctp_sort,
                                                   columns=train_x.columns)
    activation_output[output_layer] = output_train
    pred_y_df[output_layer] = get_prediction(output_test[output_layer],
                                                 pd.get_dummies(train_y['diagnosis']),
                                                 test_x,
                                                 datatype='diagnosis')
    
pred_test_y = pd.DataFra000me(data=0, index=test_x.columns, columns=['diagnosis'])
pred_test_y['diagnosis'] = pred_y_df.T.describe().T['top']

pred_train_y = pd.DataFrame(data=0, index= train_x.columns, columns=['diagnosis'])
pred_train_y['diagnosis'] = train_y_df.T.describe().T['top']'''

In [ ]:
train_y

In [ ]:
import torch

# Example tensors
x = torch.randn(1024, 216, 1)  # Shape: [1024, 216, 1]
mask = torch.randn(1024, 216, 159)  # Shape: [1024, 216, 159]

# Perform element-wise multiplication
result = mask * x  # Shape: [1024, 216, 159]

# Print shapes
print("x shape:", x.shape)
print("mask shape:", mask.shape)
print("result shape:", result.shape)



In [ ]:
labels = torch.randint(0, 2, (100,))  
labels

In [ ]:
mask.shape

In [ ]:
result

In [108]:
import torch
import torch.nn as nn

# Define a simple linear layer
layer = nn.Linear(3, 2)  # Linear layer with 3 input features and 2 output features

# Input tensor
x = torch.tensor([[1.0, 2.0, 3.0]])

# Apply the layer
layer_output = layer(x)

# Apply the tanh activation function
tanh_output = torch.tanh(layer_output)

print("Layer output:", layer_output)
print("After tanh activation:", tanh_output)

Layer output: tensor([[ 1.8656, -0.9858]], grad_fn=<AddmmBackward0>)
After tanh activation: tensor([[ 0.9532, -0.7556]], grad_fn=<TanhBackward0>)


In [109]:
layer_output.shape

torch.Size([1, 2])

In [ ]:
sum(pred_train_y.diagnosis.values == rlabel.diagnosis.values)/len(pred_train_y)

In [ ]:
sum(pred_test_y.diagnosis.values == qlabel.diagnosis.values)/len(pred_test_y )

In [ ]:
qlabel

In [110]:
import torch
import torch.nn as nn

# Define a simple linear layer
layer = nn.Linear(3, 2)  # Linear layer with 3 input features and 2 output features

# Input tensor
x = torch.tensor([[1.0, 2.0, 3.0]])

# Apply the layer
layer_output = layer(x)

# Apply the tanh activation function
tanh_output = torch.tanh(layer_output)

print("Layer output:", layer_output)
print("After tanh activation:", tanh_output)

Layer output: tensor([[0.0911, 0.1891]], grad_fn=<AddmmBackward0>)
After tanh activation: tensor([[0.0908, 0.1869]], grad_fn=<TanhBackward0>)


In [111]:
layer_output

tensor([[0.0911, 0.1891]], grad_fn=<AddmmBackward0>)

In [117]:
import torch
import torch.nn as nn

# Step 1: Define the Linear Layer
layer = nn.Linear(3, 2)

# Initialize layer weights for demonstration
nn.init.constant_(layer.weight, 5.0)
nn.init.constant_(layer.bias, 0.0)

print("Original weights:")
print(layer.weight)

# Step 2: Create a Mask Matrix
# Example mask matrix: 2x3, matching the shape of layer.weight
mask = torch.tensor([[1, 0, 1],
                     [0, 1, 0]], dtype=torch.float32)

print("Mask matrix:")
print(mask)

# Step 3: Apply the Mask
# Element-wise multiplication of the layer's weights and the mask
layer.weight.data *= mask

print("Sparse weights:")
print(layer.weight)

# Now you can use the layer in your model as usual.


Original weights:
Parameter containing:
tensor([[5., 5., 5.],
        [5., 5., 5.]], requires_grad=True)
Mask matrix:
tensor([[1., 0., 1.],
        [0., 1., 0.]])
Sparse weights:
Parameter containing:
tensor([[5., 0., 5.],
        [0., 5., 0.]], requires_grad=True)


In [120]:
masking[0].shape

(28, 135)

In [112]:
layer.weight

Parameter containing:
tensor([[-0.5483,  0.3901,  0.0170],
        [ 0.1705,  0.3766, -0.3806]], requires_grad=True)

In [113]:
layer.weight.shape

torch.Size([2, 3])

In [114]:
x.shape

torch.Size([1, 3])

In [115]:
-0.1207*1 + 0.5761*2 -0.0801*3

0.7911999999999999

In [ ]:
-0.1207,  0.5761, -0.0801

In [ ]:
(-0.3824*1) +  (0.5135*2) -(0.1200*3)

In [ ]:
import torch
import torch.nn as nn

# Define a simple linear layer
layer = nn.Linear(3, 2, bias = False)  # Linear layer with 3 input features and 2 output features

# Input tensor
x = torch.tensor([[1.0, 2.0, 3.0]])

# Apply the layer
layer_output = layer(x)

# Apply the tanh activation function
tanh_output = torch.tanh(layer_output)

print("Layer output:", layer_output)
print("After tanh activation:", tanh_output)

In [ ]:
layer.weight.shape

In [ ]:
x.shape

In [ ]:
# What nn.linear does

torch.matmul(x,layer.weight.T)    


In [ ]:
layer.weight

In [ ]:
1*0.5236 -0.5384*2-0.4127*3

In [ ]:
layer_output

In [ ]:
import torch

# Create a batch of tensors
num_samples = 1024
num_features = 3
x_batch = torch.randn(num_samples, num_features)  # Shape: [1024, 3]§

# Print the shape and content of the batch
print("x_batch shape:", x_batch.shape)
print("x_batch tensor:", x_batch)


In [ ]:
layer = nn.Linear(3, 2, bias = False) 

In [ ]:
layer.weight

In [ ]:
layer_output = layer(x_batch)

In [ ]:
layer_output 

In [ ]:
1.3851 * -0.1641 + 0.2640 * 0.5494 + 0.1831 * -1.0035